## PROYECTO 3 - MODELO

In [52]:


import sys
assert sys.version_info >= (3, 7)

from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

import tensorflow as tf
assert version.parse(tf.__version__) >= version.parse("2.8.0")

import pandas as pd

import mlflow
import mlflow.keras
import keras
import tensorflow as tf
import tensorflow.keras as tk
from keras import models
from keras import layers

import os



In [53]:
datos = pd.read_csv("datosModelo.csv")

In [54]:
datos.head()

,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global,año,estu_tipodocumento_CC,estu_tipodocumento_CE,estu_tipodocumento_CR,...,desemp_ingles_B1,semestre_1,semestre_2,semestre_4,año_nacimiento,cole_cod_dane_establecimiento_encoded,cole_cod_dane_sede_encoded,cole_cod_mcpio_ubicacion_encoded,estu_cod_mcpio_presentacion_encoded,estu_cod_reside_mcpio_encoded
0,53.0,62.0,53.0,51.0,54.0,274.0,2015,False,False,False,...,False,False,True,False,1999,256.552815,256.552815,253.403924,270.286881,253.708284
1,68.0,72.0,69.0,66.0,72.0,348.0,2014,False,False,False,...,False,False,True,False,1997,275.371382,273.762494,278.796528,270.286881,278.376939
2,49.0,56.0,49.0,51.0,60.0,268.0,2016,False,False,False,...,False,False,True,False,1999,271.668806,271.668806,283.446840,280.093872,283.943115
3,52.0,68.0,52.0,54.0,64.0,295.0,2016,False,False,False,...,False,False,True,False,2000,244.459308,244.459308,244.459308,254.947960,244.459308
4,40.0,59.0,28.0,45.0,51.0,227.0,2019,False,False,False,...,False,False,False,True,2002,249.823691,249.823691,273.910278,273.099522,257.853831


In [55]:
datos.columns

Index(['punt_ingles', 'punt_matematicas', 'punt_sociales_ciudadanas',
       'punt_c_naturales', 'punt_lectura_critica', 'punt_global', 'año',
       'estu_tipodocumento_CC', 'estu_tipodocumento_CE',
       'estu_tipodocumento_CR',
       ...
       'desemp_ingles_B1', 'semestre_1', 'semestre_2', 'semestre_4',
       'año_nacimiento', 'cole_cod_dane_establecimiento_encoded',
       'cole_cod_dane_sede_encoded', 'cole_cod_mcpio_ubicacion_encoded',
       'estu_cod_mcpio_presentacion_encoded', 'estu_cod_reside_mcpio_encoded'],
      dtype='object', length=153)

In [56]:
varscategoricas = datos.select_dtypes(include=['object', 'bool']).columns
varscategoricas

Index(['estu_tipodocumento_CC', 'estu_tipodocumento_CE',
       'estu_tipodocumento_CR', 'estu_tipodocumento_NES',
       'estu_tipodocumento_PC', 'estu_tipodocumento_PE',
       'estu_tipodocumento_PEP', 'estu_tipodocumento_PPT',
       'estu_tipodocumento_RC', 'estu_tipodocumento_TI',
       ...
       'fami_tienelavadora_No', 'fami_tienelavadora_Si', 'desemp_ingles_A-',
       'desemp_ingles_A1', 'desemp_ingles_A2', 'desemp_ingles_B+',
       'desemp_ingles_B1', 'semestre_1', 'semestre_2', 'semestre_4'],
      dtype='object', length=140)

In [57]:
varsnumericas = numeric_cols = datos.select_dtypes(include=['int64', 'float64']).columns
varsnumericas

Index(['punt_ingles', 'punt_matematicas', 'punt_sociales_ciudadanas',
       'punt_c_naturales', 'punt_lectura_critica', 'punt_global', 'año',
       'año_nacimiento', 'cole_cod_dane_establecimiento_encoded',
       'cole_cod_dane_sede_encoded', 'cole_cod_mcpio_ubicacion_encoded',
       'estu_cod_mcpio_presentacion_encoded', 'estu_cod_reside_mcpio_encoded'],
      dtype='object')

In [58]:
X = X = datos.drop(columns=['punt_global', 'punt_ingles', 'punt_matematicas', 'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica']) # Excluimos las variables de puntajes individuales
y = datos['punt_global']

In [59]:
# Dividir en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:
# Normalizar los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [61]:
# Construir el modelo

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [62]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [63]:
# Definir Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [48]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87884 entries, 0 to 87883
Columns: 153 entries, punt_ingles to estu_cod_reside_mcpio_encoded
dtypes: bool(140), float64(11), int64(2)
memory usage: 20.5 MB


In [64]:
# Iniciar una nueva corrida de MLFlow
with mlflow.start_run() as run:
    # Entrenar el modelo
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])
    
    # Evaluar el modelo
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Mean Absolute Error on Test Set: {mae}')
    
    # Loggear los parámetros y métricas en MLFlow
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss", "mean_squared_error")
    mlflow.log_param("epochs", 100)
    mlflow.log_param("batch_size", 32)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("loss", loss)
    
    # Guardar el modelo usando Keras
    model_path = "models/modelProy3"
    os.makedirs(model_path, exist_ok=True)
    model.save(os.path.join(model_path, "model.h5"))
    
    # Loggear el modelo en MLFlow
    mlflow.keras.log_model(model, "modelProy3")

MissingConfigException: Yaml file 'C:\proyecto3_ACTD\Modelos\mlruns\0\meta.yaml' does not exist.